# NOVEMBER COMPETITION ON KAGGLE

In [ ]:
import numpy as np
import pandas as pd
import re
import os
import gc # сборщик мусора
import math
import matplotlib.pyplot as plt
import matplotlib.image as img
import seaborn as sns
%matplotlib inline

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Reading Data

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
train.head()

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
train.head()

### Размерность данных

In [ ]:
train.shape

In [ ]:
test.shape

### Пропуски в данных

In [ ]:
train.isna().sum().sum()


In [ ]:
test.isna().sum().sum()

### Балансировка классов

In [ ]:
train['target'].value_counts()

### Типы данных

In [ ]:
for i in train.dtypes:
      print(i)

## Убираем мусор

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
#                 elif

                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
train = reduce_mem_usage(train)

In [ ]:
test = reduce_mem_usage(test)

### Строим гистограмму важности признаков

In [ ]:
def plot_feature_importances(importances, X):
    
    indices = np.argsort(importances)[::-1]

    plt.figure(figsize = (20, 6))
    plt.title("Feature importances", fontsize=16)
    plt.bar(range(X.shape[1]), importances[indices] / importances.sum(),
           color="darkblue", align="center")
    plt.xticks(range(X.shape[1]), X.columns[indices], rotation = 90, fontsize=14)
    plt.xlim([-1, X.shape[1]])

    plt.tight_layout()
    # plt.savefig('fe.jpg')
    plt.show()

### Информация о данных

In [ ]:
train.info()

In [ ]:
X.hist(figsize=(16,16), bins=100);

### Корреляции

In [ ]:
plt.figure(figsize = (15,10))

sns.set(font_scale=1.4)

corr_matrix = train.corr()
#print(X.corr())
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.1] = 0  # Проверьте, что будет если убрать маленькие корреляции

sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='coolwarm')

plt.title('Correlation matrix')
plt.show()

In [ ]:
corr_with_target = train.corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(10, 8))

sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

### Дропаем признаки

In [ ]:
drop_features = ['f9', 'f16', 'f19', 'f20', 'f23', 'f24', 'f30', 'f31', 'f32', 'f33', 'f42', 'f46', 'f48', 'f51', 'f56', 'f58', 'f59',
                'f61', 'f62', 'f64', 'f69', 'f72', 'f75', 'f76', 'f78', 'f88', 'f89', 'f90', 'f92', 'f93', 'f94', 'f98', 'f99']

In [ ]:
train_dropped = train.drop(drop_features, axis=1)

In [ ]:
train_dropped.head

In [ ]:
test_dropped = test.drop(drop_features, axis=1)

In [ ]:
# Get train data without the target and ids
X = train_dropped.iloc[:, 1:-1].copy()
# Get the target
y = train_dropped.target.copy()

# Create test X, drop ids.
test_X = test_dropped.iloc[:, 1:].copy()

# Обучение и подбор модели

In [ ]:
# Выделим векотр признаков, вектор целевой переменной
X = train.drop(['target', 'id'], axis=1)
y = train['target']
test = test.drop(['id'], axis=1)

X

In [ ]:
y

# Нормализация и стандартизация

In [ ]:
#СТАНДАРТИЗАЦИЯ
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
# Fit on the training data
scaler.fit(X_important)
# Transform both the training and testing data
X_standar = scaler.transform(X_important)
X_standar_f = scaler.transform(X_important_f)
print(X_standar)

In [ ]:
#Нормализация
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Create the scaler object with a range of 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
# Fit on the training data
scaler.fit(X_standar)
# Transform both the training and testing data
X_norm = scaler.transform(X_standar)
X_norm_f = scaler.transform(X_standar_f)
print(X_norm)

### Выбираем модель

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

####Метод опорных векторов

In [ ]:
from sklearn.svm import SVC
SVC_model = SVC()
SVC_model.fit(X_train, y_train)
SCV_predict = SVC_model.predict(X_test)

print(accuracy_score(y_test, y_pred))

#### Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=1000)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

In [ ]:
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

In [ ]:
clf_f = clf.RandomForestClassifier(n_estimators=1000)
clf_f.fit(X, y) #X_norm
y_pred_test = clf_f.predict(X_standar_f) # X_norm_f

#### Экстремальный градиентный бустинг

In [ ]:
#%%time
import xgboost as xgb
from sklearn.metrics import classification_report

xgb1 = xgb.XGBClassifier(max_depth = 3, min_child_weight = 0.05, learning_rate = 0.607)
xgb1.fit(X_train, y_train)
y_pred_test = xgb1.predict(X_test)
print(accuracy_score(y_test, y_pred_test))

In [ ]:
xgb_f = xgb.XGBClassifier(max_depth = 3, min_child_weight = 0.05, learning_rate = 0.607)
xgb_f.fit(X, y) #X_norm
y_pred_test = xgb_f.predict(test) # X_norm_f

#### Catboost

In [ ]:
!pip install catboost

In [ ]:
#%%time
from catboost import CatBoostClassifier
catboost = CatBoostClassifier(task_type='GPU', n_estimators=200, learning_rate=0.1, #l2_leaf_reg = 2.5, 
                              depth = 3)#500 0.1
catboost.fit(X_train, y_train)
y_pred_test = catboost.predict(X_test)
#y_test.columns
print(accuracy_score(y_test, y_pred_test))

In [ ]:
catboost_f = CatBoostClassifier(task_type='GPU', n_estimators=10000)
catboost_f.fit(X, y)
y_pred_test = catboost_f.predict(test)

In [ ]:
X_important.shape

In [ ]:
X_important_f.shape

#### Логистическая регрессия

In [ ]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(#max_depth=2,
                      max_depth=11,#11 
                             n_estimators=326,#326
                             random_state=53,
                            #objective = 'gamma',#gamma
                           # min_data_in_leaf = 27)#27)
)
model.fit(X_train, y_train)
y_pred_test = model.predict(X_test)
print(accuracy_score(y_test, y_pred_test))

In [ ]:
model_f = LGBMClassifier(#max_depth=2,
                      max_depth=11,#11 
                             n_estimators=326,#326
                             random_state=53,
                            #objective = 'gamma',#gamma
                            #min_data_in_leaf = 27)#27)
model_f.fit(X_norm, y)
y_pred_test = model_f.predict(X_norm_f)

#### Градиентный бустинг

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
y_pred_test = gbc.predict(X_test)
print(accuracy_score(y_test, y_pred_test))


### Важность признаков

In [ ]:
plot_feature_importances(importances = catboost.feature_importances_, X=X_important)

#### Убираем неважные признаки

In [ ]:
model = catboost
X_important = X.copy()
X_important_f = test.copy()
k = 0
col = []
for a in X.columns:
    col.append(a)
#print(col)
#os_data_X_1 = os_data_X
for i in model.feature_importances_:
    if i < 0.5:
        #print(col[k])
        print(i)
        X_important = X_important.drop([col[k]], axis=1)
        X_important_f = X_important_f.drop([col[k]], axis=1)
    
    k += 1

### Нейронная сеть

In [ ]:
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
network = models.Sequential()
network.add(layers.Dense(units=15, activation="relu", input_shape=(100,)))
network.add(layers.Dense(units=15, activation="selu"))
#network.add(layers.Dense(units=10, activation="relu"))
network.add(layers.Dense(units=4, activation="selu"))
#network.add(layers.Dense(units=1, activation="tanh"))
network.add(layers.Dense(units=1, activation="sigmoid"))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(
    monitor='accuracy', 
    min_delta=0, 
    patience=100, 
    verbose=0,
    mode='min', 
    baseline=None, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='accuracy', 
    factor=0.2,
    patience=200,
    mode='min'
)

In [ ]:
network.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
history = network.fit(
    X_train,
    y_train,
    epochs=18,
    verbose=1,
    batch_size=100,
    validation_data=(X_test, y_test),
        callbacks=[
            early_stopping,
            reduce_lr,
        ])

In [ ]:
history = network.fit(
    X,
    y,
    epochs=18,
    verbose=1,
    batch_size=100,
        callbacks=[
            early_stopping,
            reduce_lr,
        ])

In [ ]:
y_pred_test = network.predict(test)

### Формируем ответ

In [ ]:
indexs = [i for i in range(600000, 1140000)]

In [ ]:
test_pred = pd.DataFrame(indexs, columns=['id'])
test_pred['target'] = y_pred_test
test_pred

In [ ]:
test_pred.to_csv(r'211130_Vasenkov_16.csv', index=None)